In [1]:
import pandas as pd
import re
import itertools
import networkx as nx
from networkx.algorithms import community
from operator import itemgetter

The following code reads in the data `film_festivals.csv`. The data contains information about films selected to compete at the Cannes, Venice, and Berlin film festivals. In particular, it contains both festival-specific information—which includes which festival the film competed in, the year of the festival, and whether the film won—and film-specific information—which includes the film's title and the people involved in the making of the film.

`film_festivals.csv` was produced by joining `festivals.csv` and `films.csv`. To obtain the data for `festivals.csv`, I scraped the Wikipedia pages associated with each year of the Cannes, Venice, and Berlin festivals, using Beautiful Soup, which returned all of the films selected for competition at these festivals. To obtain the data for `films.csv`, I used the Open Movie Database (OMDb) API to request the film-specific information for each of the films listed in `festivals.csv`.

In [2]:
df = pd.read_csv('./data/film_festivals.csv', keep_default_na=False)

The following code removes missing data, encoded as 'N/A', '', or 'Unknown' values, within the `Director`, `Writer`, `Actors`, and `Production` features, specifically removing films that are missing all four. There are not many films with all of these values missing, so removing these films is possible.

Films with only one, two, or three of these features missing—but not all missing—are still kept within our data. We do not throw out these films because they still have some data on the people or entities involved in their making. We want to keep these people in our network, so we will also keep the remaining missing values for now and handle them a little differently at a later point in our analysis.

After considering the missing data, we also do some initial cleaning. For the `Director` and `Writer` features, we use regular expressions to remove pattern matches for any characters found between parentheses so that what we are left with are just the names. For example, for `Director`, a possible pattern match to be removed is '(co-director), and for `Writer`, possible pattern matches include '(screenplay)', '(dialogue)', and '(story)'.

The `Production` feature is especially messy, so we have to do some additional cleaning. We first change the value of 'Parts and Labor' to 'Parts & Labor'. Films with more than one production company have different ways of specifying this, with some using a forward slash between the companies and others using the word 'and'; we will substitute them with commas so that they are consistent with films with multiple directors, writers, or actors.

In addition to this, the `Production` feature also includes many typos or various wordings for the same production company. In order to consolidate all of these, we create a dictionary to map them to the correct wording for the company.

In [3]:
df = df[~(df['Director'].isin(['N/A', '']) & df['Writer'].isin(['N/A', '']) & df['Actors'].isin(['N/A', '']) & df['Production'].isin(['N/A', '', 'Unknown']))]
df['Director'] = df['Director'].apply(lambda row: re.sub('\(.*?\)', '', row))
df['Writer'] = df['Writer'].apply(lambda row: re.sub(' \(.*?\)', '', row))
df['Production'] = df['Production'].replace({'Parts and Labor': 'Parts & Labor'})
df['Production'] = df['Production'].apply(lambda row: re.sub('(?<![N0-9])\/', ', ', row)) # substitutes any string that has a forward slash (as long as not preceded by an N or a number) with a comma
df['Production'] = df['Production'].apply(lambda row: re.sub(' and ', ', ', row)) # substitutes any string that has ' and ' with a comma

dict_production = {'20th Century Fox': '20th Century Studios',
                   '20th Century Fox Film Corporat': '20th Century Studios',
                   '20th Century Fox Film Corporation': '20th Century Studios',
                   'Fox': '20th Century Studios',
                   'Twentieth Century Fox': '20th Century Studios',
                   'Twentieth Century Fox Home Entertainment': '20th Century Studios',
                   'A24 Films': 'A24',
                   'AMLF': 'Pathé',
                   'Pathe International': 'Pathé',
                   'Pathé Distribution': 'Pathé',
                   'ARP Films': 'ARP',
                   'AVCO Embassy Pictures': 'Embassy Pictures',
                   'Embassy Home Entertainment': 'Embassy Pictures',
                   'Embassy Pictures Corporation': 'Embassy Pictures',
                   'Allied Artists Pictures': 'Allied Artists',
                   'Monogram': 'Allied Artists',
                   'American International Picture': 'American International Pictures',
                   'Anchor Bay Entertainment': 'Anchor Bay',
                   'Anchor Bay Films': 'Anchor Bay',
                   'Starmaker Entertainment': 'Anchor Bay',
                   'Artificial Eye': 'Curzon Artificial Eye',
                   'Artificial Eye Film Co. Ltd': 'Curzon Artificial Eye',
                   'Curzon Artificial Eye': 'Curzon Artificial Eye',
                   'Curzon Film Distributors': 'Curzon Artificial Eye',
                   'Artisan Home Entertainment': 'Artisan Entertainment',
                   'Artisan Pictures': 'Artisan Entertainment',
                   'Live Home Video': 'Artisan Entertainment',
                   'Bavaria Film International': 'Bavaria Film',
                   'Bavaria-Filmkunst': 'Bavaria Film',
                   'Buena Vista Distribution Compa': 'Buena Vista',
                   'Buena Vista Pictures': 'Buena Vista',
                   'Cinema Service': 'CJ Entertainment',
                   'Centre National de la Cinématographie': 'CNC',
                   'Cannon Releasing': 'Cannon Films',
                   'Golan-Globus Productions': 'Cannon Films',
                   'Castle Hill Productions, First Run': 'Castle Hill Productions, First Run Features',
                   'Channel IV': 'Channel 4',
                   'Fox Searchlight': 'Searchlight Pictures',
                   'Fox Searchlight Pictures': 'Searchlight Pictures',
                   'Fox Searchlight Pictures, Film4, Waypoint': 'Searchlight Pictures, Channel 4, Waypoint',
                   'Christal Films': 'Entertainment One',
                   'Koch': 'Entertainment One',
                   'Koch Lorber Films': 'Entertainment One',
                   'Koch Vision': 'Entertainment One',
                   'Cinema 5': 'Cinema 5 Distributing',
                   'Cinema V': 'Cinema 5 Distributing',
                   'Cinema Libre': 'Cinema Libre Studio',
                   'Cinépix Film Properties Inc.': 'Lionsgate',
                   'Liongate Films': 'Lionsgate',
                   'Lions Gate': 'Lionsgate',
                   'Lions Gate Films': 'Lionsgate',
                   'Lions Gate Films Inc.': 'Lionsgate',
                   'Lions Gate Releasing': 'Lionsgate',
                   'LionsGate Entertainment': 'Lionsgate',
                   'City Lights': 'City Lights Pictures',
                   'Cohen Film Collection': 'Cohen Media Group',
                   'Cohen Media': 'Cohen Media Group',
                   'Columbia': 'Columbia Pictures',
                   'Columbia Pictures Corporation': 'Columbia Pictures',
                   'Columbia Pictures of Brasil': 'Columbia Pictures',
                   'RCA, Columbia': 'RCA, Columbia Pictures',
                   'Sony': 'Sony Pictures',
                   'Sony Classic Pictures': 'Sony Pictures',
                   'Sony Classics': 'Sony Pictures',
                   'Sony Picture Classics': 'Sony Pictures',
                   'Sony Pictures Classics': 'Sony Pictures',
                   'Sony Pictures Entertainment': 'Sony Pictures',
                   'Sony Pictures Home Entertainment': 'Sony Pictures',
                   'TriStar Pictures': 'Sony Pictures',
                   'Sony, Columbia': 'Sony Pictures, Columbia Pictures',
                   'Edward Harrison, Sony Pictures Classics': 'Edward Harrison, Sony Pictures',
                   'Concorde, New Horizons Home Video': 'New Concorde',
                   'New Concorde Home Entertainment': 'New Concorde',
                   'Continental': 'Continental Distributing',
                   'Continental Distributing Inc.': 'Continental Distributing',
                   'Continental Motion Pictures': 'Continental Distributing',
                   'Cowboy Booking International': 'Cowboy',
                   'Cowboy Pictures': 'Cowboy',
                   'Diaphana Films': 'Diaphana',
                   'The Weinstein Co.': 'The Weinstein Company',
                   'Radius-TWC': 'The Weinstein Company',
                   'The Weinstein Co., Dimension': 'The Weinstein Company, Dimension Films',
                   'DisneyToon Studios': 'Disney',
                   'Walt Disney': 'Disney',
                   'Walt Disney Pictures': 'Disney',
                   'Walt Disney Productions': 'Disney',
                   'DreamWorks SKG': 'DreamWorks Pictures',
                   'Dreamworks': 'DreamWorks Pictures',
                   'Dreamworks Pictures': 'DreamWorks Pictures',
                   'Go Fish Pictures': 'DreamWorks Pictures',
                   'Europa Corp': 'EuropaCorp',
                   'EuropaCorp. Distribution': 'EuropaCorp',
                   'Excelsior Film TV~RAI 1': 'Excelsior Film TV, RAI',
                   '01 Distribution': 'RAI',
                   'RAI Cinema': 'RAI',
                   'RAI Radiotelevisione Italiana': 'RAI',
                   'Fine Line': 'New Line Cinema',
                   'Fine Line Features': 'New Line Cinema',
                   'New Line Home Entertainment': 'New Line Cinema',
                   'First Look Pictures': 'First Look Studios',
                   'Overseas Filmgroup': 'First Look Studios',
                   'Flach Pyramide International': 'Pyramide Films',
                   'Pyramide Distribution': 'Pyramide Films',
                   'Fortissimo Film Sales': 'Fortissimo Films',
                   'Fortissimo World Sales': 'Fortissimo Films',
                   'Films Antenne 2': 'France Télévisions',
                   'France 3 Cinéma': 'France Télévisions',
                   'General Film Distributors': 'J. Arthur Rank Productions',
                   'Golden Way Films': 'Golden Harvest Company',
                   'Gramercy': 'Gramercy Pictures',
                   'HBO Films': 'HBO',
                   'HBO Video': 'HBO',
                   'Home Box Office': 'HBO',
                   'IFC': 'IFC Films',
                   'IFC First Take': 'IFC Films',
                   'Sundance Selects': 'IFC Films',
                   'IFC Films, Sundance Selects': 'IFC Films',
                   'Image Entertainment Inc.': 'Image Entertainment',
                   'Joseph Burstyn Incorporated': 'Joseph Burstyn Inc.',
                   'Kimstim': 'Kimstim Films',
                   'Kino': 'Kino Lorber',
                   'Kino International': 'Kino Lorber',
                   'Kino International Corp.': 'Kino Lorber',
                   'Kino Lorber Films': 'Kino Lorber',
                   'Kino on Video': 'Kino Lorber',
                   'Lorber': 'Kino Lorber',
                   'Lorber Films': 'Kino Lorber',
                   'Leisure Time': 'Leisure Time Features',
                   'Lolafilms Distribucion': 'LolaFilms', 
                   'Lux': 'Lux Film',
                   'Lux Productions': 'Lux Film',
                   'MCA, Universal Pictures': 'MCA, Universal',
                   'Universal Focus': 'Universal',
                   'Universal International Pictur': 'Universal',
                   'Universal Pictures': 'Universal',
                   'Universal Studios': 'Universal',
                   'Universal Studios Home Video': 'Universal',
                   'MGM Distribution Co': 'MGM',
                   'MGM Distribution Company': 'MGM',
                   'MGM Home Entertainment': 'MGM',
                   'Metro-Goldwyn-Mayer (MGM)': 'MGM',
                   'Metro-Goldwyn-Mayer Pictures': 'MGM',
                   'United Artists Films': 'United Artists',
                   'United Artists Pictures': 'United Artists',
                   'MGM, UA': 'MGM, United Artists',
                   'MK2 Diffusion': 'MK2',
                   'MK2 Productions': 'MK2',
                   'Magnet Releasing': 'Magnolia Pictures',
                   'Mars Distribution': 'Mars Films',
                   'Medusa': 'Medusa Film',
                   'Medusa Distribuzione': 'Medusa Film',
                   'Miramax Films': 'Miramax',
                   'Miramax Zoe': 'Miramax',
                   'New Yorker': 'New Yorker Films',
                   'October Film': 'October Films',
                   'October, Universal': 'October Films, Universal',
                   'Orion Classics': 'Orion Pictures',
                   'Orion Home Video': 'Orion Pictures',
                   'Orion Pictures Corporation': 'Orion Pictures',
                   'Oscilloscope Laboratories': 'Oscilloscope Pictures',
                   'Pacific Arts Video': 'Pacific Arts',
                   'Palm': 'Palm Pictures',
                   'Palomar Pictures International': 'Palomar Pictures',
                   'Paramount Classics': 'Paramount',
                   'Paramount Home Video': 'Paramount',
                   'Paramount Pictures': 'Paramount',
                   'Paramount Vantage': 'Paramount',
                   'RKO Radio Pictures': 'RKO Pictures',
                   'Republic': 'Republic Pictures',
                   'Republic Pictures Corporation': 'Republic Pictures',
                   'Republic Pictures Home Video': 'Republic Pictures',
                   'Rialto': 'Rialto Pictures',
                   'Senator Entertainment AG': 'Wild Bunch',
                   'Senator Film': 'Wild Bunch',
                   'Sigma III Corp.': 'Sigma III',
                   'Strand': 'Strand Releasing',
                   'Tartan USA': 'Tartan Films',
                   'Shooting Gallery': 'The Shooting Gallery',
                   'ThinkFilm Inc.': 'ThinkFilm',
                   'Titanus Films': 'Titanus',
                   'Titanus Produzione': 'Titanus',
                   'Tribecca Film': 'Tribeca Film',
                   'Triumph Releasing Corporation': 'Triumph Films',
                   'VCI': 'VCI Entertainment',
                   'WARNER BROTHERS PICTURES': 'Warner Bros.',
                   'WB': 'Warner Bros.',
                   'Warner Bros. Pictures': 'Warner Bros.',
                   'Warner Home Video': 'Warner Bros.',
                   'Warner Independent Pictures': 'Warner Bros.',
                   'Water Bearer Films Inc.': 'Water Bearer Films',
                   'Well Go USA Inc.': 'Well Go USA',
                   'Wellspring': 'Wellspring Media',
                   'Wellspring Media Inc.': 'Wellspring Media',
                   'Winstar': 'Winstar Cinema',
                   'Winstar Cinemas': 'Winstar Cinema',
                   'Zebra Film': 'Zebra Films',
                   'Zeitgeist': 'Zeitgeist Films',
                   'Zenith International Films': 'Zenith Entertainment',
                   'Zespol Filmowy Droga': 'Zespol Filmowy',
                   'ABC': 'ABC Weekend TV',
                   'Amplify Releasing': 'Amplify',
                   'Astor Pictures Corporation': 'Astor Pictures',
                   'Bac Films': 'BAC Films',
                   'Bandai Visual Company': 'Bandai Visual',
                   'Big Beach Films': 'Big Beach',
                   'Bo Films !': 'Bo Films',
                   'Emerging Picutres': 'Emerging Pictures',
                   'Le Studio Canal Plus': 'StudioCanal',
                   'MCA Universal Home Video': 'MCA',
                   'Picture House': 'Picturehouse'}

df['Production'] = df['Production'].replace(dict_production)

df = df.reset_index(drop=True)
df.head()

,id,title_english,title_original,director,country,winner,festival,year_festival,link_film,link_director,...,BoxOffice,Production,Website,Response,InternetMovieDatabaseRating,RottenTomatoesRating,MetacriticRating,Error,totalSeasons,Ratings
0,0,Adieu Bonaparte,وداعا بونابرت,Youssef Chahine,Egypt,0,cannes,1985,https://en.wikipedia.org/wiki/Adieu_Bonaparte,https://en.wikipedia.org/wiki/Youssef_Chahine,...,N/A,N/A,N/A,True,6.4/10,N/A,N/A,,,
1,1,Birdy,N/A,Alan Parker,United States,0,cannes,1985,https://en.wikipedia.org/wiki/Birdy_(film),https://en.wikipedia.org/wiki/Alan_Parker,...,N/A,Sony Pictures,N/A,True,7.3/10,85%,N/A,,,
2,2,Bliss,N/A,Ray Lawrence,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/Bliss_(1985_film),https://en.wikipedia.org/wiki/Ray_Lawrence_(fi...,...,N/A,Anchor Bay,N/A,True,6.9/10,N/A,N/A,,,
3,4,The Coca-Cola Kid,N/A,Dušan Makavejev,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/The_Coca-Cola_Kid,https://en.wikipedia.org/wiki/Du%C5%A1an_Makav...,...,N/A,Cinecom Pictures,N/A,True,6.0/10,44%,N/A,,,
4,5,Colonel Redl,Oberst Redl,István Szabó,Hungary,0,cannes,1985,https://en.wikipedia.org/wiki/Colonel_Redl,https://en.wikipedia.org/wiki/Istv%C3%A1n_Szab...,...,N/A,N/A,N/A,True,7.6/10,N/A,N/A,,,


The idea behind the node list is to create a list of all of the nodes involved in the network. In the context of the networks that we will form for film festivals, we are interested in nodes for the network consisting of the people and entities involved in the making of the films: the directors, writers, actors, and production companies. The function `get_nodelist` creates this node list.

`get_nodelist` takes in strings consisting of the values for the director, writer, actor, and production company. In order to prepare these strings to form the node list, each of these strings are split by commas in order to get lists consisting of individual people or entities. We then remove any remaining missing values—encoded as 'N/A' for directors, writers, and actors, and as 'N/A', '', or 'Unknown' for production companies—because we do not want to include them as nodes in our network. We also add a descriptor to each of the names, describing their roles in their films (e.g. 'director', 'writer', 'actor', 'production company'). We combine each of these individual lists into one total list, which we then turn into our node list dataframe. As a final step, we remove any duplicate names. The reasoning for this is that the node list only keeps track of what nodes should exist in the network, but not the number of times they appear in various films, which should instead emerge from the edge list; hence, we only need each name to appear once in our node list.

In [4]:
def get_nodelist(directors_from_df, writers_from_df, actors_from_df, production_companies_from_df):
    '''
    Given strings containing the directors, writers, actors, and production companies
    involved in the making of the films to be included in the network,
    return the node list. For each node, the node list also includes the attribute
    of the role the node played in the making of the film, e.g. 'director',
    'writer', 'actor', 'production company'.
    
    Keyword arguments:
    directors_from_df -- string containing directors for all of the films, separated by commas
    writers_from_df -- string containing writers for all of the films, separated by commas
    actors_from_df -- string containing actors for all of the films, separated by commas
    production_companies_from_df -- string containing production companies for all of the films, separated by commas
    '''
    # directors
    directors = directors_from_df.split(', ')
    directors = [director for director in directors if director != 'N/A']
    directors = [(director, 'director') for director in directors]
    # writers
    writers = writers_from_df.split(', ')
    writers = [writer for writer in writers if writer != 'N/A']
    writers = [(writer, 'writer') for writer in writers]
    # actors
    actors = actors_from_df.split(', ')
    actors = [actor for actor in actors if actor != 'N/A']
    actors = [(actor, 'actor') for actor in actors]
    # production companies
    production_companies = production_companies_from_df.split(', ')
    production_companies = [production_company for production_company in production_companies if production_company not in ['N/A', '', 'Unknown']]
    production_companies = [(production_company, 'production company') for production_company in production_companies]
    
    # people: directors, actors, writers, and production companies
    people = directors
    people.extend(actors) # the order of the extends determines the order of priority: director -> actor -> writer for people who have multiple roles
    people.extend(writers)
    people.extend(production_companies)
    
    name = [person[0] for person in people]
    role = [person[1] for person in people]
    nodelist = pd.DataFrame({'Name': name,
                             'Role': role})
    nodelist = nodelist.drop_duplicates(subset='Name').reset_index(drop=True)
    return nodelist

The idea behind the edge list is to create a list of all of the edges between two nodes. An edge refers to a relation between nodes; in the  case of films, this relation exists between each of the people involved in the making of a film. As an additional note, in the context of films, the network is undirected because the relations are symmetric: Person A made the film with Person B, and Person B made the film with Person A—the edges are reciprocal and symmetric. The function `get_edgelist` creates the edge list.

`get_edgelist` takes in a node list. From the node list, we know all of the people involved in the making of a film; all of these people should have an edge between them because they all worked together on the film. `get_edgelist` creates all of the combinations of people without repeat values. Each edge is then given a weight of 1, referring to the film they worked on.

In [5]:
def get_edgelist(nodelist):
    '''
    Given a node list, returns a list of edges, both as a list and as a dataframe.
    The edge list is created by taking combinations of each node in the list
    without any repeat values. Each edge is given a weight of 1.
    
    Keyword arguments:
    nodelist -- node list returned by `get_nodelist` function
    '''
    edges = [combo + (1,) for combo in itertools.combinations(nodelist['Name'], 2)]
    edgelist = pd.DataFrame(edges, columns=['Source', 'Target', 'Weight'])
    return edges, edgelist

The `get_film_edges` function takes in strings consisting of the values for the director, writer, actor, and production company. `get_film_edges` creates the node list and the corresponding edge list—from that node list—all in one function.

In [6]:
def get_film_edges(directors_from_df, writers_from_df, actors_from_df, production_companies_from_df):
    '''
    Given strings containing the directors, writers, actors, and production companies
    involved in the making of the films to be included in the network,
    creates the node list and the corresponding edge list from that node list.
    Returns the list of edges, both as a list and as a dataframe.
    
    Keyword arguments:
    directors_from_df -- string containing directors for all of the films, separated by commas
    writers_from_df -- string containing writers for all of the films, separated by commas
    actors_from_df -- string containing actors for all of the films, separated by commas
    production_companies_from_df -- string containing production companies for all of the films, separated by commas
    '''
    nodelist_film = get_nodelist(directors_from_df,
                                 writers_from_df,
                                 actors_from_df,
                                 production_companies_from_df)
    edges_film, edgelist_film = get_edgelist(nodelist_film)
    return edges_film

The function `get_graph` takes in the overall dataframe of films selected to compete in the festivals. It first subsets the data to get the relevant data of interest for our network analysis by filtering on the festival and years of the festival. The network will be built on this subset. Then, for each of the features `Director`, `Writer`, `Actors`, and `Production`, we create a string consisting of all of the values for director, writer, actor, and production company, respectively, separated by a comma.

Network analysis requires a node list and an edge list. From the strings, we are able to create the node list for our entire network. It includes nodes for every person and entity involved in the making of all of the films included in our subset.

Getting the edges for our network is a bit more complicated. Unlike the node list, we cannot just create the edge list for our network all in one go. Because edges only exist between people who worked together on the same films, we cannot just make an edge list using `get_edgelist` based on the overall node list containing nodes from all of the films; instead, we need to look at each film individually. Hence, for each row in our data—which corresponds to an individual film—we use `get_film_edges` to create a miniature node list just for that film and then use it to get its corresponding edge list. After doing this for each film, we can then bring each of the edge lists together to get the overall edge list for our network. For people who worked together on multiple films, duplicate entries will be included in the edge list.

Keep in mind that, at this point, each of the edges has a weight of 1—including each of the duplicate entries. We will handle these weights once we've created our graphs.

To create the graph, we use the NetworkX package in Python. We first start with a MultiGraph—an undirected graph class in NetworkX—to be able to handle the duplicate entries. In graph theory, a multigraph is a graph that can have multiple edges between the same nodes. Hence, all of the duplicate entries in our edge list manifest as multiple edges in our MultiGraph.

We can then convert the MultiGraph into a weighted Graph so that we can change the multiple edges into a single edge with weights.

In [7]:
def get_graph(df, festival, from_year, to_year):
    '''
    Returns graph/network associated with the subset of data taken by filtering on
    festival and years of the festival.
    
    Keyword arguments:
    df -- dataframe
    festival -- string for which festival, e.g. 'cannes', 'venice', or 'berlin'
    from_year -- the first year in a range of festival years, used to subset the data, inclusive
    to_year -- the last year in a range of festival years, used to subset the data, inclusive
    '''
    df = df[(df['festival'] == festival) & (df['year_festival'] >= from_year) & (df['year_festival'] <= to_year)].reset_index(drop=True)
    
    df_directors = df['Director'].str.cat(sep=', ')
    df_writers = df['Writer'].str.cat(sep=', ')
    df_actors = df['Actors'].str.cat(sep=', ')
    df_production_companies = df['Production'].str.cat(sep=', ')
    
    df_nodelist = get_nodelist(df_directors,
                               df_writers,
                               df_actors,
                               df_production_companies)
    df_node_names = list(df_nodelist['Name'])

    df_edges = df.apply(lambda row: get_film_edges(row['Director'], row['Writer'], row['Actors'], row['Production']), axis=1)
    df_edges = df_edges.tolist()
    df_edges = [item for sublist in df_edges for item in sublist]
    
    M = nx.MultiGraph()
    M.add_nodes_from(df_node_names)
    M.add_weighted_edges_from(df_edges)
    
    # https://stackoverflow.com/questions/15590812/networkx-convert-multigraph-into-simple-graph-with-weighted-edges
    # create weighted graph from M
    G = nx.Graph() 
    for u,v,data in M.edges(data=True):
        w = data['weight'] if 'weight' in data else 1.0
        if G.has_edge(u,v):
            G[u][v]['weight'] += w
        else:
            G.add_edge(u, v, weight=w)
    
    return G

In [8]:
def get_centrality_metrics(G):

    degree_dict = dict(G.degree(G.nodes())) # Run degree centrality
    betweenness_dict = nx.betweenness_centrality(G) # Run betweenness centrality
    eigenvector_dict = nx.eigenvector_centrality(G, max_iter=1000) # Run eigenvector centrality

    # Assign each to an attribute in your network
    nx.set_node_attributes(G, degree_dict, 'degree')
    nx.set_node_attributes(G, betweenness_dict, 'betweenness')
    nx.set_node_attributes(G, eigenvector_dict, 'eigenvector')

    sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)
    print("Top 20 nodes by degree:")
    for d in sorted_degree[:20]:
        print(d)

    print('\n')

    sorted_betweenness = sorted(betweenness_dict.items(), key=itemgetter(1), reverse=True)
    print("Top 20 nodes by betweenness centrality:")
    for b in sorted_betweenness[:20]:
        print(b)

    print('\n')

    sorted_eigenvector = sorted(eigenvector_dict.items(), key=itemgetter(1), reverse=True)
    print("Top 20 nodes by eigenvector centrality:")
    for e in sorted_eigenvector[:20]:
        print(e)

In [9]:
G_cannes_2018_2019 = get_graph(df, 'cannes', 2018, 2019)
get_centrality_metrics(G_cannes_2018_2019)

Top 20 nodes by degree:
('Sony Pictures', 18)
('Magnolia Pictures', 17)
('Focus Features', 14)
('Matteo Garrone', 11)
('Marcello Fonte', 11)
('Edoardo Pesce', 11)
('Nunzia Schiano', 11)
('Adamo Dionisi', 11)
('Ugo Chiti', 11)
('Massimo Gaudioso', 11)
('Marco Perfetti', 11)
("Damiano D'Innocenzo", 11)
("Fabio D'Innocenzo", 11)
('Giulio Troli', 11)
('Spike Lee', 9)
('Alec Baldwin', 9)
('John David Washington', 9)
('Isiah Whitlock Jr.', 9)
('Robert John Burke', 9)
('Charlie Wachtel', 9)


Top 20 nodes by betweenness centrality:
('Sony Pictures', 0.0023156089193825044)
('Magnolia Pictures', 0.0018867924528301887)
('Focus Features', 0.0012864493996569467)
('Fan Liao', 0.00017152658662092623)
("Yi'nan Diao", 0.00017152658662092623)
('Jafar Panahi', 0.0)
('Behnaz Jafari', 0.0)
('Marziyeh Rezaei', 0.0)
('Maedeh Erteghaei', 0.0)
('Nader Saeivar', 0.0)
('Kino Lorber', 0.0)
('Ryûsuke Hamaguchi', 0.0)
('Masahiro Higashide', 0.0)
('Erika Karata', 0.0)
('Sairi Itô', 0.0)
('Kôji Nakamoto', 0.0)
('Tom

In [10]:
G_venice_2018_2019 = get_graph(df, 'venice', 2018, 2019)
get_centrality_metrics(G_venice_2018_2019)

Top 20 nodes by degree:
('Netflix', 26)
('IFC Films', 22)
('Joaquin Phoenix', 16)
('Jack London', 14)
('Amazon Studios', 13)
('Mario Martone', 10)
('Ippolita Di Majo', 10)
('Juliette Binoche', 10)
('Yorgos Lanthimos', 9)
('Olivia Colman', 9)
('Rachel Weisz', 9)
('Emma Delves', 9)
('Faye Daveney', 9)
('Deborah Davis', 9)
('Tony McNamara', 9)
('Searchlight Pictures', 9)
('Channel 4', 9)
('Waypoint', 9)
('Olivier Assayas', 9)
('Todd Phillips', 9)


Top 20 nodes by betweenness centrality:
('Netflix', 0.011085379033503165)
('IFC Films', 0.010313416705264783)
('Jack London', 0.004817044928207503)
('Juliette Binoche', 0.004014204106839586)
('Olivier Assayas', 0.00333487725798981)
('Joaquin Phoenix', 0.0019453450671607225)
('Amazon Studios', 0.0012351397251814111)
('Mario Martone', 0.0003087849312953528)
('Ippolita Di Majo', 0.0003087849312953528)
('Gonzalo Tobal', 0.0)
('Lali Espósito', 0.0)
('Leonardo Sbaraglia', 0.0)
('Inés Estévez', 0.0)
('Daniel Fanego', 0.0)
('Ulises Porra', 0.0)
('Julia

In [11]:
G_berlin_2018_2019 = get_graph(df, 'berlin', 2018, 2019)
get_centrality_metrics(G_berlin_2018_2019)

Top 20 nodes by degree:
('Franz Rogowski', 11)
('Gus Van Sant', 8)
('Joaquin Phoenix', 8)
('Jonah Hill', 8)
('Rooney Mara', 8)
('Jack Black', 8)
('John Callahan', 8)
('Jack Gibson', 8)
('William Andrew Eatman', 8)
('Amazon Studios', 8)
('Wes Anderson', 8)
('Bryan Cranston', 8)
('Koyu Rankin', 8)
('Edward Norton', 8)
('Bob Balaban', 8)
('Roman Coppola', 8)
('Jason Schwartzman', 8)
('Kunichi Nomura', 8)
('Scott Rudin Productions', 8)
('Cédric Kahn', 7)


Top 20 nodes by betweenness centrality:
('Franz Rogowski', 0.0014347202295552368)
('Emily Atef', 0.0)
('Marie Bäumer', 0.0)
('Birgit Minichmayr', 0.0)
('Charly Hübner', 0.0)
('Robert Gwisdek', 0.0)
('David Zellner', 0.0)
('Nathan Zellner', 0.0)
('Robert Pattinson', 0.0)
('Mia Wasikowska', 0.0)
('Magnolia Pictures', 0.0)
('Gus Van Sant', 0.0)
('Joaquin Phoenix', 0.0)
('Jonah Hill', 0.0)
('Rooney Mara', 0.0)
('Jack Black', 0.0)
('John Callahan', 0.0)
('Jack Gibson', 0.0)
('William Andrew Eatman', 0.0)
('Amazon Studios', 0.0)


Top 20 nodes

In [12]:
G_cannes_2016_2017 = get_graph(df, 'cannes', 2016, 2017)
get_centrality_metrics(G_cannes_2016_2017)

Top 20 nodes by degree:
('IFC Films', 20)
('Elle Fanning', 15)
('Isabelle Huppert', 13)
('Marion Cotillard', 13)
('A24', 12)
('Amazon Studios', 12)
('Min-hee Kim', 11)
('Adèle Haenel', 11)
('Netflix', 11)
('Cohen Media Group', 10)
('Paul Verhoeven', 8)
('Laurent Lafitte', 8)
('Anne Consigny', 8)
('Charles Berling', 8)
('Philippe Djian', 8)
('David Birke', 8)
('Harold Manning', 8)
('SBS Productions', 8)
('Nicole Garcia', 8)
('Louis Garrel', 8)


Top 20 nodes by betweenness centrality:
('IFC Films', 0.006772329184838492)
('Marion Cotillard', 0.0035457220863028757)
('Elle Fanning', 0.0019856043683296105)
('Isabelle Huppert', 0.0014182888345211503)
('A24', 0.0012764599510690351)
('Amazon Studios', 0.0012764599510690351)
('Adèle Haenel', 0.0010637166258908627)
('Netflix', 0.0010637166258908627)
('Min-hee Kim', 0.0009928021841648052)
('Cohen Media Group', 0.0008864305215757189)
('Andrea Arnold', 0.0)
('Sasha Lane', 0.0)
('Shia LaBeouf', 0.0)
('Riley Keough', 0.0)
('McCaul Lombardi', 0.0)
('P

In [13]:
G_venice_2016_2017 = get_graph(df, 'venice', 2016, 2017)
get_centrality_metrics(G_venice_2016_2017)

Top 20 nodes by degree:
('Searchlight Pictures', 17)
('Amy Adams', 13)
('Michael Shannon', 12)
('Jean-Pierre Darroussin', 11)
('Paramount', 11)
('A24', 11)
('Weiwei Ai', 8)
('Israa Abboud', 8)
('Hiba Abed', 8)
('Rami Abu Sondos', 8)
('Chin-Chin Yap', 8)
('Tim Finch', 8)
('Boris Cheshirkov', 8)
('Amazon Studios', 8)
('Participant Media', 8)
('Paolo Virzì', 8)
('Helen Mirren', 8)
('Donald Sutherland', 8)
('Christian McKay', 8)
('Janel Moloney', 8)


Top 20 nodes by betweenness centrality:
('Searchlight Pictures', 0.007803317226067651)
('Michael Shannon', 0.007215619694397284)
('Amy Adams', 0.005256627922162727)
('Jean-Pierre Darroussin', 0.0009794958861172784)
('Paramount', 0.0009794958861172784)
('A24', 0.0009794958861172784)
('Denis Villeneuve', 0.0)
('Jeremy Renner', 0.0)
('Forest Whitaker', 0.0)
('Michael Stuhlbarg', 0.0)
('Eric Heisserer', 0.0)
('Ted Chiang', 0.0)
('21 Laps Entertainment', 0.0)
('Ana Lily Amirpour', 0.0)
('Suki Waterhouse', 0.0)
('Jason Momoa', 0.0)
('Jayda Fink', 0

In [14]:
G_berlin_2016_2017 = get_graph(df, 'berlin', 2016, 2017)
get_centrality_metrics(G_berlin_2016_2017)

Top 20 nodes by degree:
('João Pedro Vaz', 10)
('Vincent Perez', 8)
('Emma Thompson', 8)
('Daniel Brühl', 8)
('Brendan Gleeson', 8)
('Mikael Persbrandt', 8)
('Achim von Borries', 8)
('Bettine von Borries', 8)
('Hans Fallada', 8)
('X-Filme Creative Pool', 8)
('Michael Grandage', 7)
('Colin Firth', 7)
('Jude Law', 7)
('Nicole Kidman', 7)
('Laura Linney', 7)
('John Logan', 7)
('A. Scott Berg', 7)
('Riverstone Pictures', 7)
('Georg Friedrich', 7)
('Ivo Ferreira', 6)


Top 20 nodes by betweenness centrality:
('João Pedro Vaz', 0.0010629816635663034)
('Georg Friedrich', 0.0005314908317831517)
('Denis Côté', 0.0)
('James Hyndman', 0.0)
('Simone-Élise Girard', 0.0)
('Denis Lavant', 0.0)
('Isolda Dychauk', 0.0)
('Michael Grandage', 0.0)
('Colin Firth', 0.0)
('Jude Law', 0.0)
('Nicole Kidman', 0.0)
('Laura Linney', 0.0)
('John Logan', 0.0)
('A. Scott Berg', 0.0)
('Riverstone Pictures', 0.0)
('Vincent Perez', 0.0)
('Emma Thompson', 0.0)
('Daniel Brühl', 0.0)
('Brendan Gleeson', 0.0)
('Mikael Pers

In [15]:
G_cannes_2014_2015 = get_graph(df, 'cannes', 2014, 2015)
get_centrality_metrics(G_cannes_2014_2015)

Top 20 nodes by degree:
('Sony Pictures', 48)
('IFC Films', 27)
('The Weinstein Company', 16)
('Marion Cotillard', 15)
('Vincent Cassel', 15)
('A24', 13)
('Thomas Bidegain', 13)
('Louis Garrel', 12)
('Isabelle Huppert', 12)
('Lionsgate', 11)
('Hsiao-Hsien Hou', 9)
('Qi Shu', 9)
('Chen Chang', 9)
('Yun Zhou', 9)
('Satoshi Tsumabuki', 9)
('Cheng Ah', 9)
("T'ien-wen Chu", 9)
('Hai-Meng Hsieh', 9)
('Xing Pei', 9)
('Central Motion Pictures Corporation', 9)


Top 20 nodes by betweenness centrality:
('Sony Pictures', 0.10196991162382785)
('IFC Films', 0.07550765701949673)
('Thomas Bidegain', 0.061424812790933016)
('Marion Cotillard', 0.04317614517978817)
('The Weinstein Company', 0.021014639411725024)
('Louis Garrel', 0.016545233758348512)
('Vincent Cassel', 0.013661202185792351)
('A24', 0.0014167172637117994)
('Isabelle Huppert', 0.0012143290831815423)
('Lionsgate', 0.0010119409026512851)
('Atom Egoyan', 0.0)
('Kevin Durand', 0.0)
('Alexia Fast', 0.0)
('Mireille Enos', 0.0)
('Rosario Dawson'

In [16]:
G_venice_2014_2015 = get_graph(df, 'venice', 2014, 2015)
get_centrality_metrics(G_venice_2014_2015)

Top 20 nodes by degree:
('IFC Films', 18)
('Maurizio Braucci', 15)
('Alba Rohrwacher', 11)
('Francesco Munzi', 8)
('Marco Leonardi', 8)
('Peppino Mazzotta', 8)
('Fabrizio Ferracane', 8)
('Barbora Bobulova', 8)
('Gioacchino Criaco', 8)
('Fabrizio Ruggirello', 8)
('Vitagraph Films', 8)
('Luca Guadagnino', 8)
('Tilda Swinton', 8)
('Matthias Schoenaerts', 8)
('Ralph Fiennes', 8)
('Dakota Johnson', 8)
('David Kajganich', 8)
('Jacques Deray', 8)
('Alain Page', 8)
('Jean-Claude Carrière', 8)


Top 20 nodes by betweenness centrality:
('IFC Films', 0.005354040350317342)
('Alba Rohrwacher', 0.0025529199021380703)
('Maurizio Braucci', 0.0019856043683296105)
('Ramin Bahrani', 0.0)
('Michael Shannon', 0.0)
('Douglas M. Griffin', 0.0)
('Randy Austin', 0.0)
('Carl Palmer', 0.0)
('Bahareh Azimi', 0.0)
('Amir Naderi', 0.0)
('Broad Green Pictures', 0.0)
('Guy Bolongaro', 0.0)
('Caroline Christl Long', 0.0)
('Searchlight Pictures', 0.0)
('Francesco Munzi', 0.0)
('Marco Leonardi', 0.0)
('Peppino Mazzotta'

In [17]:
G_berlin_2014_2015 = get_graph(df, 'berlin', 2014, 2015)
get_centrality_metrics(G_berlin_2014_2015)

Top 20 nodes by degree:
('IFC Films', 16)
('Kino Lorber', 13)
('Florian Stetter', 11)
('Rachid Bouchareb', 9)
('Harvey Keitel', 9)
('Forest Whitaker', 9)
('Ellen Burstyn', 9)
('Brenda Blethyn', 9)
('Olivier Lorelle', 9)
('Yasmina Khadra', 9)
('José Giovanni', 9)
('Daniel Boulanger', 9)
('Cohen Media Group', 9)
('Alain Resnais', 8)
('Sabine Azéma', 8)
('Sandrine Kiberlain', 8)
('Caroline Silhol', 8)
('André Dussollier', 8)
('Alan Ayckbourn', 8)
('Laurent Herbiet', 8)


Top 20 nodes by betweenness centrality:
('IFC Films', 0.0030138637733574444)
('Kino Lorber', 0.0015601177179732652)
('Florian Stetter', 0.0010637166258908627)
('Yann Demange', 0.0)
("Jack O'Connell", 0.0)
('Jack Lowden', 0.0)
('Paul Popplewell', 0.0)
('Adam Nagaitis', 0.0)
('Gregory Burke', 0.0)
('Roadside Attractions', 0.0)
('Alain Resnais', 0.0)
('Sabine Azéma', 0.0)
('Sandrine Kiberlain', 0.0)
('Caroline Silhol', 0.0)
('André Dussollier', 0.0)
('Alan Ayckbourn', 0.0)
('Laurent Herbiet', 0.0)
('Jean-Marie Besset', 0.0)


In [18]:
G_cannes_2012_2013 = get_graph(df, 'cannes', 2012, 2013)
get_centrality_metrics(G_cannes_2012_2013)

Top 20 nodes by degree:
('IFC Films', 46)
('The Weinstein Company', 24)
('Sony Pictures', 23)
('Kino Lorber', 17)
('Strand Releasing', 17)
('Mads Mikkelsen', 13)
('Marion Cotillard', 13)
('Matthew McConaughey', 12)
('Mathieu Amalric', 12)
('Isabelle Huppert', 10)
('Yuh Jung Youn', 10)
('Matteo Garrone', 8)
('Aniello Arena', 8)
('Loredana Simioli', 8)
('Nando Paone', 8)
('Nello Iorio', 8)
('Massimo Gaudioso', 8)
('Maurizio Braucci', 8)
('Ugo Chiti', 8)
('Oscilloscope Pictures', 8)


Top 20 nodes by betweenness centrality:
('IFC Films', 0.10124080882352941)
('Sony Pictures', 0.0975796568627451)
('Isabelle Huppert', 0.08927696078431373)
('Yuh Jung Youn', 0.07169117647058823)
('Marion Cotillard', 0.0625)
('The Weinstein Company', 0.053462009803921566)
('Kino Lorber', 0.03056066176470588)
('Mathieu Amalric', 0.015165441176470588)
('Strand Releasing', 0.0029411764705882353)
('Mads Mikkelsen', 0.001286764705882353)
('Matthew McConaughey', 0.0011029411764705882)
('Yusri Nasrullah', 0.0)
('Menn

In [19]:
G_venice_2012_2013 = get_graph(df, 'venice', 2012, 2013)
get_centrality_metrics(G_venice_2012_2013)

Top 20 nodes by degree:
('The Weinstein Company', 21)
('Toni Servillo', 14)
('Emerging Pictures', 13)
('Davide Lantieri', 13)
('Rachel McAdams', 12)
('James Franco', 12)
('A24', 12)
('Sony Pictures', 11)
('Amplify', 11)
('Philippe Garrel', 8)
('Louis Garrel', 8)
('Anna Mouglalis', 8)
('Rebecca Convenant', 8)
('Olga Milshtein', 8)
('Marc Cholodenko', 8)
('Caroline Deruas-Garrel', 8)
('Arlette Langmann', 8)
('DistriB Films', 8)
('Marco Bellocchio', 7)
('Isabelle Huppert', 7)


Top 20 nodes by betweenness centrality:
('Emerging Pictures', 0.007095239951658805)
('The Weinstein Company', 0.0061206190791781995)
('Toni Servillo', 0.005457876885891388)
('Davide Lantieri', 0.005457876885891388)
('James Franco', 0.003274726131534833)
('A24', 0.003274726131534833)
('Rachel McAdams', 0.001364469221472847)
('Sony Pictures', 0.001169545046976726)
('Amplify', 0.001169545046976726)
('Ramin Bahrani', 0.0)
('Dennis Quaid', 0.0)
('Kim Dickens', 0.0)
('Aaron B. Oduber', 0.0)
('Jacob R. Oduber', 0.0)
('Hal

In [20]:
G_berlin_2012_2013 = get_graph(df, 'berlin', 2012, 2013)
get_centrality_metrics(G_berlin_2012_2013)

Top 20 nodes by degree:
('Adopt Films', 25)
('Isabelle Huppert', 15)
('Léa Seydoux', 13)
('Gilles Taurand', 13)
('Cohen Media Group', 13)
('Magnolia Pictures', 13)
('Nina Hoss', 11)
('Brillante Mendoza', 8)
('Katherine Mulville', 8)
('Marc Zanetta', 8)
('Rustica Carpio', 8)
('Patrick Bancarel', 8)
('Boots Agbayani Pastor', 8)
('Arlyn Dela Cruz', 8)
('Vancouver Film School', 8)
('Antonio Chavarrías', 8)
('Juan Diego Botto', 8)
('Bárbara Lennie', 8)
('Mágica Pérez', 8)
('Marc Rodríguez', 8)


Top 20 nodes by betweenness centrality:
('Adopt Films', 0.023340289048612718)
('Cohen Media Group', 0.008346858335265476)
('Nina Hoss', 0.007148929592704227)
('Léa Seydoux', 0.0067238581034083)
('Gilles Taurand', 0.0067238581034083)
('Isabelle Huppert', 0.002164000309142901)
('Magnolia Pictures', 0.001623000231857176)
('Melody Schaal', 0.0)
('Bear Schaal', 0.0)
('Christian Petzold', 0.0)
('Ronald Zehrfeld', 0.0)
('Rainer Bock', 0.0)
('Christina Hecke', 0.0)
('Harun Farocki', 0.0)
('Paolo Taviani', 0

In [21]:
G_cannes_2010_2011 = get_graph(df, 'cannes', 2010, 2011)
get_centrality_metrics(G_cannes_2010_2011)

Top 20 nodes by degree:
('IFC Films', 44)
('Sony Pictures', 23)
('Lambert Wilson', 14)
('Strand Releasing', 12)
('Hafsia Herzi', 12)
('Nanni Moretti', 12)
('Michel Piccoli', 12)
('Jerzy Stuhr', 12)
('Renato Scarpa', 12)
('Franco Graziosi', 12)
('Francesco Piccolo', 12)
('Federica Pontremoli', 12)
('Tom King', 12)
('Jer Lujan', 12)
('Rayan Desaintanger', 12)
('Georgii Nik', 12)
('Georgia Molotta', 12)
('The Weinstein Company', 11)
('Sean Penn', 11)
('Kino Lorber', 10)


Top 20 nodes by betweenness centrality:
('IFC Films', 0.04963235294117647)
('Lambert Wilson', 0.031004901960784315)
('Sony Pictures', 0.028952205882352942)
('The Weinstein Company', 0.0016850490196078432)
('Sean Penn', 0.0016850490196078432)
('Strand Releasing', 0.0011029411764705882)
('Hafsia Herzi', 0.0010723039215686275)
('Kino Lorber', 0.0007659313725490196)
('Magnolia Pictures', 0.0006127450980392157)
('Mike Leigh', 0.0)
('Jim Broadbent', 0.0)
('Ruth Sheen', 0.0)
('Lesley Manville', 0.0)
('Oliver Maltman', 0.0)
('Al

In [22]:
G_venice_2010_2011 = get_graph(df, 'venice', 2010, 2011)
get_centrality_metrics(G_venice_2010_2011)

Top 20 nodes by degree:
('Sony Pictures', 32)
('Vincent Cassel', 15)
('Searchlight Pictures', 15)
('Michael Fassbender', 14)
('Oscilloscope Pictures', 13)
('Kino Lorber', 13)
('Focus Features', 13)
('Harold Manning', 13)
('Paul Giamatti', 12)
('Independent Pictures', 12)
('Luca Marinelli', 12)
('IFC Films', 12)
('Ariane Labed', 10)
('Yorgos Lanthimos', 10)
('Vincent Gallo', 10)
('Johnnie To', 10)
('Ching Wan Lau', 10)
('Richie Jen', 10)
('Denise Ho', 10)
('Myolie Wu', 10)


Top 20 nodes by betweenness centrality:
('Sony Pictures', 0.02104514752932812)
('Michael Fassbender', 0.009325749496385828)
('Harold Manning', 0.006541059367223605)
('Vincent Cassel', 0.005593079748785402)
('Searchlight Pictures', 0.002215902358099301)
('Kino Lorber', 0.0016589643322668564)
('Oscilloscope Pictures', 0.0009953785993601137)
('Focus Features', 0.0009479796184382037)
('Independent Pictures', 0.0008531816565943833)
('Luca Marinelli', 0.0008531816565943833)
('IFC Films', 0.0008294821661334282)
('Ariane La

In [23]:
G_berlin_2010_2011 = get_graph(df, 'berlin', 2010, 2011)
get_centrality_metrics(G_berlin_2010_2011)

Top 20 nodes by degree:
('IFC Films', 13)
('Kino Lorber', 13)
('Olive Films', 12)
('Kim Fupz Aakeson', 12)
('Film Movement', 12)
('Roadside Attractions', 9)
('Independent Pictures', 9)
('Oskar Roehler', 9)
('Tobias Moretti', 9)
('Martina Gedeck', 9)
('Moritz Bleibtreu', 9)
('Justus von Dohnányi', 9)
('Klaus Richter', 9)
('Franz Novotny', 9)
('Friedrich Knilli', 9)
('Michael Esser', 9)
('Concorde Filmverleih', 9)
('Béla Tarr', 7)
('Ágnes Hranitzky', 7)
('János Derzsi', 7)


Top 20 nodes by betweenness centrality:
('IFC Films', 0.0018087855297157622)
('Kino Lorber', 0.0018087855297157622)
('Olive Films', 0.0015503875968992248)
('Kim Fupz Aakeson', 0.0015503875968992248)
('Film Movement', 0.0015073212747631353)
('Roadside Attractions', 0.0008613264427217916)
('Independent Pictures', 0.0008613264427217916)
('Seyfi Teoman', 0.0)
('Ilker Aksum', 0.0)
('Fatih Al', 0.0)
('Gunes Sayin', 0.0)
('Taner Birsel', 0.0)
('Baris Bicakci', 0.0)
('unafilm', 0.0)
('Ralph Fiennes', 0.0)
('Gerard Butler', 0